In [ ]:
import time
import json
import nest_asyncio
import asyncio
import websockets
import numpy as np
import math
import random
import colorsys
from tqdm.notebook import tqdm
nest_asyncio.apply()

In [ ]:
async def websocket_test(payload, indent=None, quiet=False):
    uri = "ws://127.0.0.1:8765/ws"
    async with websockets.connect(uri) as ws:
        await ws.send(json.dumps(payload))
        if not quiet:
            print("✅ send:", payload)

        try:
            # 5 秒以内に返事が来なければ TimeoutError
            response = await asyncio.wait_for(ws.recv(), timeout=5)
            if not quiet:
                if indent:
                    print("📨 recv: ", json.dumps(json.loads(response), indent=indent, ensure_ascii=False))
                else:
                    print("📨 recv:", response)
            return json.loads(response)
        except asyncio.TimeoutError:
            print("⏰ タイムアウト（5 秒以内にメッセージが来ませんでした）")

In [12]:
payload = {
    "type": "tree",
}
tree = await websocket_test(payload)

for item in tree:
    if item["name"] == "center":
        payload = {
            "type": "delete",
            "id": item["objectInfo"]["id"],
        }
        await websocket_test(payload)

payload = {
    "type": "add",
    "command": "folder",
    "name": "center"
}
response = await websocket_test(payload)
center_id = response["objectId"]

payload = {
    "type": "update",
    "command": "transform",
    "id": center_id,
    "pos": [0.0, 1.5, 0.5],
}
response = await websocket_test(payload)

✅ send: {'type': 'tree'}
📨 recv: [{"name":"篠崎 遥","objectInfo":{"id":0,"type":"OCICharFemale"},"children":[]},{"name":"center","objectInfo":{"id":2212,"type":"OCIFolder"},"children":[{"name":"キューブ(通常）","objectInfo":{"id":2213,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2215,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2217,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2219,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2221,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2223,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2225,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2227,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2229,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2231,"type":"OCIItem"},"children":[]},{"name":"キューブ(通常）","objectInfo":{"id":2233,"type":"OCIItem

In [13]:

def random_color_with_fixed_saturation(saturation=1.0, value=1.0):
    h = random.uniform(0.0, 1.0)
    r, g, b = colorsys.hsv_to_rgb(h, saturation, value)
    return [r, g, b, 1.0]

RANGE = 5

for angle in tqdm(range(360*3)):
    if angle % 10 != 0:
        continue

    payload = {
        "type": "add",
        "command": "item",
        "group": 0,
        "category": 0,
        "itemId": 1,
    }
    response = await websocket_test(payload, quiet=True)
    item_id = response["objectId"]

    payload = {
        "type": "hierarchy",
        "command": "attach",
        "childId": item_id,
        "parentId": center_id,
    }
    response = await websocket_test(payload, quiet=True)

    range_ = random.uniform(0.1, 5)
    height = random.uniform(-1.5, 1.5)
    scale = random.uniform(1, 3)
    payload = {
        "type": "update",
        "command": "transform",
        "id": item_id,
        "pos": [
            math.cos(math.radians(angle))*range_,
            height,
            math.sin(math.radians(angle))*range_,
        ],
        "rot": [random.uniform(0, 360) for _ in range(3)],
        "scale": [scale for _ in range(3)],
    }
    response = await websocket_test(payload, quiet=True)

    payload = {
        "type": "update",
        "command": "color",
        "id": item_id,
        "color": list(random_color_with_fixed_saturation(0.9, 0.9)),
        "colorIndex": 0,
    }
    response = await websocket_test(payload, quiet=True)

  0%|          | 0/1080 [00:00<?, ?it/s]